In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyllama-empathetic-sft-v8/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v8/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v8/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v8/README.md
/kaggle/input/tinyllama-empathetic-sft-v8/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v8/tokenizer_config.json
/kaggle/input/tinyllama-empathetic-sft-v8/special_tokens_map.json
/kaggle/input/tinyllama-empathetic-sft-v8/tokenizer.model
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/adapter_model.safetensors
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/trainer_state.json
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/training_args.bin
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/adapter_config.json
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/README.md
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/tokenizer.json
/kaggle/input/tinyllama-empathetic-sft-v8/checkpoint-936/tokeni

In [2]:
!pip install datasets evaluate transformers accelerate peft trl 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:01m0:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstal

In [3]:
# Cell 1: Imports and Setup
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from peft import LoraConfig
import torch


2025-06-09 21:53:40.655715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749506020.922649      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749506021.009197      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Cell 2: Load tokenizer and model

model1_path = "/kaggle/input/tinyllama-empathetic-sft-v6"
tokenizer = AutoTokenizer.from_pretrained(model1_path, use_fast=True)

base_model = AutoModelForCausalLM.from_pretrained(
    model1_path,
    device_map=None,          # disables automatic device placement
    torch_dtype=torch.float32 # use float32 for CPU compatibility
).to("cpu") 

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [5]:
model2_path = "/kaggle/input/tinyllama-empathetic-sft-v8"
tokenizer = AutoTokenizer.from_pretrained(model2_path, use_fast=True)

fine_tuned_model = AutoModelForCausalLM.from_pretrained(
    model2_path,
    device_map=None,          # disables automatic device placement
    torch_dtype=torch.float32 # use float32 for CPU compatibility
).to("cpu") 

In [6]:
pairs = [
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel like no one really understands what I’m going through.\n"
        "<|assistant|>\n",
        "I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy best friend hasn’t talked to me in days, and it’s making me anxious.\n"
        "<|assistant|>\n",
        "That sounds really tough. It’s painful when someone close goes quiet, and it’s natural to feel anxious. Remember, their silence doesn’t define your worth. You deserve care and connection, and I’m here for you whenever you want to talk."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m overwhelmed with school and feel like I’m falling behind.\n"
        "<|assistant|>\n",
        "I hear you, it’s so overwhelming when school piles up and you feel like you’re slipping behind. It’s okay to feel this way, and it doesn’t mean you’re failing. Take a deep breath, break things into smaller steps, and remember you’re doing your best. You’ve got this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI tried really hard on my project, but it didn’t turn out the way I hoped.\n"
        "<|assistant|>\n",
        "That’s really disappointing, and I’m sorry your project didn’t go as you hoped. Your effort matters so much, and setbacks don’t erase all the hard work you put in. Every experience is a step forward, even if it doesn’t feel that way now. Keep believing in yourself."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI got some bad news about a family member today.\n"
        "<|assistant|>\n",
        "I’m so sorry to hear that. Receiving tough news about someone you love can feel overwhelming and heavy. It’s okay to feel whatever comes up right now, grief, fear, confusion. You’re not alone, and I’m here to support you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI feel really lonely these days.\n"
        "<|assistant|>\n",
        "Feeling lonely can be so painful, and I’m sorry you’re going through this. Remember, your feelings matter, and it’s okay to reach out, even just a little. You’re not alone, and there are people who care deeply about you. I’m here to listen whenever you need."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nEveryone expects me to be strong, but I’m really struggling inside.\n"
        "<|assistant|>\n",
        "That’s incredibly hard, to carry the weight of others’ expectations while struggling inside. It’s okay to admit you’re not okay. Your strength includes being honest about your feelings, and you don’t have to face this alone. You deserve support and kindness."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nMy partner and I had a big argument and now I feel emotionally drained.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. Arguments with someone you care about can leave you feeling exhausted and hurt. It’s okay to take time to breathe and care for yourself. Your feelings are valid, and healing takes time. You’re not alone."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI’m scared about what the future holds. Everything feels uncertain.\n"
        "<|assistant|>\n",
        "Feeling scared about the future is completely natural when everything feels uncertain. It’s okay to not have all the answers right now. Take things one step at a time, and be gentle with yourself. You’re stronger than you realize, and I’m here with you through this."
    ),
    (
        "<|system|>\nHow would you respond to this empthatically,Respond without using emoticons.\n"
        "<|user|>\nI just feel like giving up lately.\n"
        "<|assistant|>\n",
        "I’m really sorry you’re feeling this way. It’s okay to feel overwhelmed and unsure, but please remember you’re not alone and your feelings matter. Reaching out for support can make a difference, you deserve kindness and hope, even when it’s hard to see."
    ),
]


In [9]:
def collect_model_responses_for_human_eval(pairs, base_model, fine_tuned_model, tokenizer, device):
    results = []

    # Set models to evaluation mode
    base_model.eval()
    fine_tuned_model.eval()

    for i, (prompt, expected_response) in enumerate(pairs):
        # Tokenize without truncating, no padding (to preserve prompt shape)
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        with torch.no_grad():
            # Generate from base model
            base_output = base_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)

            # Generate from fine-tuned model (DPO)
            fine_output = fine_tuned_model.generate(
                input_ids=input_ids,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
            fine_response = tokenizer.decode(fine_output[0], skip_special_tokens=True)

        results.append({
            "index": i,
            "prompt": prompt,
            "expected_response": expected_response,
            "sft6_response": base_response,
            "sft8_tuned_response": fine_response
        })

    return results


In [10]:
device = "cpu" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [12]:
human_eval_data = collect_model_responses_for_human_eval(
    pairs=pairs,
    base_model=base_model,
    fine_tuned_model=fine_tuned_model,
    tokenizer=tokenizer,
    device=device
)

In [13]:
# Example: Print nicely for human review
for item in human_eval_data:
    print(f"\n--- Example {item['index'] + 1} ---")
    print(f"Prompt:\n{item['prompt']}")
    print(f"\nExpected Response:\n{item['expected_response']}")
    print(f"\nSFT6 Response:\n{item['sft6_response']}")
    print(f"\nSFT8 Response:\n{item['sft8_tuned_response']}")
    print("\n" + "-" * 60)



--- Example 1 ---
Prompt:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>


Expected Response:
I'm really sorry you're feeling that way. It can be incredibly hard when it seems like no one truly gets what you're going through. You're not alone, and your feelings are valid. I'm here to listen and support you.

SFT6 Response:
<|system|>
How would you respond to this empthatically,Respond without using emoticons.
<|user|>
I feel like no one really understands what I’m going through.
<|assistant|>
I can understand that. People might have a hard time understanding how much it means to you. It's a good thing that you have a support system or confidants. It's not easy to go through a tough time with no one to talk to. 

as for the topic, I'd like to hear about the good times you have had. 

i know it's hard when things aren't going as planned but try to keep a positive